# Load Ikea Images

In [16]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict
import requests
import re
import json
import time

import os
os.getcwd()

from shared.image_processor import *

import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
pd.set_option('mode.chained_assignment', None)


In [17]:
with open('shared/all_urls.json', 'r') as file:
    all_image_urls = json.load(file)

print(f"Load {len(all_image_urls)} images")
# all_image_urls[0:30]
# len(all_image_urls)

Load 3195 images


# Image to Text

In [19]:
import os
import google.generativeai as genai

# Load the API key from environment variable
genai.configure(api_key=os.environ["GOOGLE_API_KEY1"])

# Define generation configuration
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

# Define the model with the detailed instruction
model = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    generation_config=generation_config,
    system_instruction='''{
        "prompt": "Describe the furniture image provided. The description should include:
    1. Style
    2. Detailed description.
    The response must be in the following format:
    {"style": "<style>","description": "<detailed description>"}",
    "response": [
        {"style": "coastal", "description": "This image depicts a rectangular sofa with a smooth fabric surface. The dominant colors are a mix of serene blue and crisp white, giving the room a fresh and airy appearance. Complementing the beach-inspired theme, the sofa is adorned with white and blue striped cushions, evoking a sense of calm and relaxation. The coastal style is emphasized by the use of light, natural wood for the legs, and the presence of seaside decor elements such as a driftwood coffee table and nautical-themed wall art."},
        {"style": "farmhouse", "description": "The furniture piece is a rectangular dining table with a rough-hewn texture, characterized by its rich brown color. The table is made from reclaimed wood, showcasing natural grain patterns and distressed finishes that add to its rustic charm. The farmhouse style is further evident through the use of sturdy, thick legs and the presence of metal accents, giving the table a vintage, yet timeless appeal. The setting is completed with mismatched wooden chairs and a centerpiece of fresh flowers, creating a warm and inviting atmosphere."},
        {"style": "bohemian", "description": "This image features a circular armchair with a vibrant, patterned upholstery that combines dominant green and brown hues. The chair's eclectic design is characterized by an artistic mix of floral and geometric patterns, reflecting a bohemian style. The armchair is complemented by an array of colorful throw pillows and a woven rug with intricate patterns, adding to the room's eclectic and artistic vibe. The surrounding decor includes macrame wall hangings, potted plants, and an assortment of vintage and handcrafted items, creating a lively and free-spirited environment."},
        {"style": "traditional", "description": "The image showcases a rectangular bed with a smooth, luxurious fabric headboard. The bed is primarily white with subtle gray accents, reflecting a traditional style. The classic design is indicated by the use of elegant, tufted details on the headboard and footboard, and the presence of matching bedside tables with intricate carvings. The bedding is layered with crisp white linens, embroidered pillows, and a plush, quilted comforter. Traditional elements such as antique lamps, framed artwork, and a richly patterned area rug complete the room, exuding a sense of timeless elegance."},
        {"style": "modern", "description": "This rectangular cabinet has a sleek, smooth surface with a deep black finish. The modern style is characterized by its clean lines, minimalistic design, and lack of ornate detailing. The cabinet features flat-front doors with discreet, push-to-open mechanisms, maintaining the minimalist aesthetic. The surrounding space is decorated with contemporary art pieces, a geometric area rug, and streamlined furniture, creating a sophisticated and uncluttered environment. Subtle accent lighting highlights the cabinet's sleek surface, adding to the modern ambiance."},
        {"style": "rustic", "description": "The image depicts a rectangular dining table with a smooth, natural wood texture, featuring warm brown and beige tones. The rustic style is reflected in the table's handcrafted appearance, with visible wood grain and knots adding to its natural charm. The table is paired with sturdy wooden benches and chairs, each with a slightly weathered finish. The setting includes rustic decor elements such as a burlap table runner, mason jar vases, and wrought iron chandeliers. The overall ambiance is cozy and inviting, perfect for a farmhouse setting."},
        {"style": "bohemian", "description": "This rectangular armchair features a vibrant, patterned upholstery with dominant yellow and white colors. The bohemian style is evident through the eclectic mix of patterns, including floral and abstract designs. The chair is adorned with an array of colorful throw pillows and a fringed blanket, enhancing its artistic and free-spirited look. The room is decorated with bohemian elements such as hanging plants, macrame wall art, and a mix of vintage and handcrafted decor items. The overall atmosphere is lively and inviting, reflecting a bohemian lifestyle."},
        {"style": "modern", "description": "The image showcases a rectangular bookshelf with a smooth, black surface. The modern style is defined by its clean lines, minimalist design, and functional form. The bookshelf features open shelves with no visible hardware, maintaining a sleek and uncluttered look. The shelves are sparsely decorated with a mix of books, modern art pieces, and metallic accents, reinforcing the contemporary aesthetic. The surrounding space includes modern furniture with sharp angles, a monochromatic color scheme, and subtle accent lighting, creating a sophisticated and stylish environment."},
        {"style": "farmhouse", "description": "This circular coffee table features a rough-hewn texture with brown and black tones. The farmhouse style is indicated by its rustic, handcrafted appearance and sturdy construction. The table is made from reclaimed wood with visible grain and natural imperfections, adding to its vintage charm. The setting includes farmhouse decor elements such as a woven basket centerpiece, mason jar candles, and distressed wooden accents. The overall atmosphere is warm and welcoming, evoking the feeling of a cozy countryside home."},
        {"style": "traditional", "description": "The image depicts a rectangular bedside table with a smooth, white and gray finish. The traditional style is reflected in its classic design, featuring elegant moldings and brass hardware. The bedside table includes a single drawer with a decorative pull and an open shelf for additional storage. The setting is completed with traditional decor items such as a vintage lamp, framed photographs, and a lace doily. The overall ambiance is serene and timeless, characteristic of a traditional bedroom."}
            ]

    }'''
)

In [20]:

images_list = []

for jpg_url in all_image_urls[0:100]:
    # Send a GET request to the page
    print(jpg_url)

    # Create a chat session with the uploaded file
    chat_session = model.start_chat(
        history=[
            {
                "role": "user",
                "parts": [
                    jpg_url,
                ],
            }
        ]
    )
    try:
        # Send the image to the model for analysis
        response = chat_session.send_message(jpg_url)

        # Clean and convert the response text to a dictionary
        cleaned_text = response.text.replace('\n', '').strip()
        image_desc = json.loads(cleaned_text)
        print(image_desc)
        
        # Append the details to the images list
        images_list.append({
            'image_url': jpg_url,
            'style': image_desc['style'],
            'image_desc': image_desc['description']
        })
    except Exception as e:
        print(e)
        time.sleep(60)
        try:
            # Send the image to the model for analysis
            response = chat_session.send_message(jpg_url)

            # Clean and convert the response text to a dictionary
            cleaned_text = response.text.replace('\n', '').strip()
            image_desc = json.loads(cleaned_text)
            print(image_desc)
            
            # Append the details to the images list
            images_list.append({
                'image_url': jpg_url,
                'style': image_desc['style'],
                'image_desc': image_desc['description']
            })
        except Exception as e:
            print(e)

https://www.ikea.com/images/an-oak-veneer-tonstad-bed-with-storage-with-kungscissus-bed--65fb3a5f2dc3bcb4bae52079dd280904.jpg


I0000 00:00:1722800457.104601 10104578 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


{'style': 'modern', 'description': 'This image showcases a Tonstad bed frame with built-in storage. The bed frame is made of oak veneer, giving it a natural and warm look, while the design is characterized by clean lines and a minimalist aesthetic, typical of modern style. The headboard is low and streamlined, and the bed frame sits on slightly elevated legs, adding to its contemporary appeal. The built-in storage compartments are discreetly hidden beneath the bed base, offering a practical storage solution without compromising the sleek appearance. '}
https://www.ikea.com/images/two-blue-kivik-armchairs-face-each-other-on-a-black-and-whit-e8eb237382f450dbf0c88bd152083b1f.jpg
{'style': 'modern', 'description': 'The image showcases two identical armchairs upholstered in a light blue fabric.  The chairs feature a rounded, contemporary design with a slightly reclined back and curved armrests. They rest on short, sturdy wooden legs that complement the blue upholstery. The overall aesthetic

In [12]:
images_list

[{'image_url': 'https://www.ikea.com/images/an-oak-veneer-tonstad-bed-with-storage-with-kungscissus-bed--65fb3a5f2dc3bcb4bae52079dd280904.jpg',
  'style': 'modern',
  'image_desc': 'This image showcases a platform bed frame with built-in storage. The bed frame is made of oak veneer, giving it a natural and warm appearance.  The headboard is low and rectangular, blending seamlessly with the base. The bed features four large drawers for storage - two on each side.  The overall design is characterized by clean lines, functionality, and a minimalist aesthetic, typical of modern furniture.'},
 {'image_url': 'https://www.ikea.com/images/two-blue-kivik-armchairs-face-each-other-on-a-black-and-whit-e8eb237382f450dbf0c88bd152083b1f.jpg',
  'style': 'modern',
  'image_desc': 'The image showcases two identical armchairs placed opposite each other, creating a cozy conversational setting. The chairs feature a clean and minimalist design with gentle curves, embodying a modern aesthetic. They are uph

In [21]:
import json

with open('data/image_desc_list0.json', 'w') as f:
  json.dump(images_list, f)